In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# load trained model, scaler, and ohe
model = load_model('model.h5')

# pickle is a de-sterialized form
with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [6]:
print(ohe_geo.categories_[0])
print(label_encoder.classes_)

['France' 'Germany' 'Spain']
['Female' 'Male']


In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:
input_data['Gender']

'Male'

In [18]:
[input_data]

[{'CreditScore': 600,
  'Geography': 'France',
  'Gender': 'Male',
  'Age': 40,
  'Tenure': 3,
  'Balance': 60000,
  'NumOfProducts': 2,
  'HasCrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 50000}]

In [19]:
pred_df = pd.DataFrame([input_data])
pred_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
label_encoder.transform(pd.Series(input_data['Gender']))

array([1])

In [21]:
pred_df['Gender'] = label_encoder.transform(pred_df['Gender'])

In [22]:
pred_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [27]:
ohe_df = pd.DataFrame(ohe_geo.transform(pred_df[['Geography']]).toarray(), columns=['Geo_France', 'Geo_Germany', 'Geo_Spain'])
pred_df = pd.concat([pred_df.drop("Geography", axis=1), ohe_df], axis=1)
pred_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geo_France,Geo_Germany,Geo_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [30]:
scaled_pred_df = scaler.transform(pred_df)
scaled_pred_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [31]:
model.predict(scaled_pred_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 121s 121s/step


array([[0.08550385]], dtype=float32)